### 1.1:A partir de la hoja 1, encontrar el patrón con el que se identifican las subcuentas de una cuenta (o subcuenta) dada.

Importacion de paqueterias y descarga de datos

In [1]:
import numpy as np
import pandas as pd

In [2]:
Ruta_Data = 'C:/Users/jesus/Downloads/Prueba'
df_cuentas = pd.read_excel(f'{Ruta_Data}/Cuentas.xlsx',sheet_name='Cuentas internas')
df_cuentas_regulatorias = pd.read_excel(f'{Ruta_Data}/Cuentas.xlsx',sheet_name='Cuentas regulatorias')
df_relacion = pd.read_excel(f'{Ruta_Data}/Cuentas.xlsx',sheet_name='Relación')
df_ejemplo = pd.read_excel(f'{Ruta_Data}/Cuentas.xlsx',sheet_name='Ejemplo de resultado')

In [3]:
# Revisamos los duplicados de cada dataframe
print(len(df_cuentas['CUENTA'].unique()) == len(df_cuentas) )
print(len(df_cuentas_regulatorias['CUENTA'].unique()) == len(df_cuentas_regulatorias) )
print(len(df_relacion['Cuenta regulatoria'].unique()) == len(df_relacion))
print(len(df_relacion['Cuenta interna'].unique()) == len(df_relacion) )

True
True
False
True


Primeras vistas, impresiones he hipotesis

In [4]:
df_cuentas

,CUENTA
0,1
1,11
2,1101
3,110101
4,110103
...,...
7163,96010101
7164,96010102
7165,960102
7166,96010201


In [5]:
df_cuentas['largo'] = df_cuentas['CUENTA'].astype('str').apply(len)
print('El minimo del largo de los elementos es:',df_cuentas['largo'].min())
print('El maximo del largo de los elementos es:',df_cuentas['largo'].max())

El minimo del largo de los elementos es: 1
El maximo del largo de los elementos es: 14


In [6]:
df_cuentas['largo'].value_counts()

largo
12    2036
10    1834
14    1468
8     1035
6      511
4      201
2       48
1        9
11       9
13       9
9        4
3        3
7        1
Name: count, dtype: int64

Con estos resultados podemos dar una primera hipótesis de que las cuentas son los números mas cortos (De 2 o 1 dígito)  y a medida que aumenta el nivel de la subcuenta se van agregando valores a la derecha, por eso es que solo hay 9 números con largo = 1, ya que ningún numero de cuenta comienza con el valor "0".
Aun así ocuparemos un cluster para ver si encuentra algo que nosotros no podemos ver


In [ ]:
df_cuentas_V2 = df_cuentas['CUENTA'].astype('str').apply(lambda x: pd.Series(list(x)))
df_cuentas_V2 = df_cuentas_V2.merge(df_cuentas,left_index=True, right_index=True)
df_cuentas_V2.replace({np.nan:0},inplace=True)
df_cuentas_V2.drop(columns='CUENTA',inplace=True)
df_cuentas_V2.rename(columns={'largo':14},inplace=True)
df_cuentas_V2 = df_cuentas_V2.astype(int)

# Modelo:

from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
df = df_cuentas_V2
# Crear el modelo DBSCAN
dbscan = DBSCAN(eps=5, min_samples=9)
# Ajustar el modelo a los datos
dbscan.fit(df)
# Obtener los clusters asignados a cada punto
df['cluster'] = dbscan.labels_
len(df['cluster'].unique())

8

In [ ]:
df['cluster'].value_counts()

cluster
 0    6808
 5      81
 4      78
 3      75
-1      73
 1      25
 2      18
 6      10
Name: count, dtype: int64

El cluster no ha detectado un patron significativo, por ello nos quedaremos con la hipotesis principal 

### 1.2	Organizar la información como en la hoja “Ejemplo de resultado” para todas las subcuentas dentro del esquema de todas las cuentas internas relacionadas a una cuenta regulatoria.

Con las conclusiones del ejercicio pasado podemos decir que hay a lo mas 7 niveles de subcuentas.

In [31]:
df_cuentas['Nivel'] = 0
df_cuentas['Nivel'] = np.where(df_cuentas['largo']<=2,'N1',df_cuentas['Nivel'])
df_cuentas['Nivel'] = np.where(df_cuentas['largo']>=3,'N2',df_cuentas['Nivel'])
df_cuentas['Nivel'] = np.where(df_cuentas['largo']>=5,'N3',df_cuentas['Nivel'])
df_cuentas['Nivel'] = np.where(df_cuentas['largo']>=7,'N4',df_cuentas['Nivel'])
df_cuentas['Nivel'] = np.where(df_cuentas['largo']>=9,'N5',df_cuentas['Nivel'])
df_cuentas['Nivel'] = np.where(df_cuentas['largo']>=11,'N6',df_cuentas['Nivel'])
df_cuentas['Nivel'] = np.where(df_cuentas['largo']>=13,'N7',df_cuentas['Nivel'])

In [32]:
df_cuentas

,CUENTA,largo,Nivel
0,1,1,N1
1,11,2,N1
2,1101,4,N2
3,110101,6,N3
4,110103,6,N3
...,...,...,...
7163,96010101,8,N4
7164,96010102,8,N4
7165,960102,6,N3
7166,96010201,8,N4


### Generarmos el cruce

In [33]:
Df_total = df_cuentas.merge(df_relacion,left_on='CUENTA',right_on='Cuenta interna',how='left')
Df_total = Df_total.merge(df_cuentas_regulatorias,left_on='Cuenta regulatoria',right_on='CUENTA',how='left')

### Generamos subconjuntos

In [34]:
df_N1 = Df_total[Df_total['Nivel'] == 'N1' ]
df_N2 = Df_total[Df_total['Nivel'] == 'N2' ]
df_N3 = Df_total[Df_total['Nivel'] == 'N3' ]
df_N4 = Df_total[Df_total['Nivel'] == 'N4' ]
df_N5 = Df_total[Df_total['Nivel'] == 'N5' ]
df_N6 = Df_total[Df_total['Nivel'] == 'N6' ]
df_N7 = Df_total[Df_total['Nivel'] == 'N7' ]

In [35]:
df_N3.head()

,CUENTA_x,largo,Nivel,Cuenta regulatoria,Cuenta interna,CUENTA_y,Descripción
3,110101,6,N3,NaN,NaN,NaN,NaN
4,110103,6,N3,NaN,NaN,NaN,NaN
5,110104,6,N3,NaN,NaN,NaN,NaN
6,110105,6,N3,NaN,NaN,NaN,NaN
7,110106,6,N3,NaN,NaN,NaN,NaN


### Generamos apartir de la subcuenta, la cuenta

In [36]:
df_N2['Cuenta_Master'] = np.where(df_N2['largo']==3,df_N2['CUENTA_x'].astype('str').str[:1],df_N2['CUENTA_x'].astype('str').str[:2])
df_N3['Cuenta_Master'] = np.where(df_N3['largo']==5,df_N3['CUENTA_x'].astype('str').str[:3],df_N3['CUENTA_x'].astype('str').str[:4])
df_N4['Cuenta_Master'] = np.where(df_N4['largo']==7,df_N4['CUENTA_x'].astype('str').str[:5],df_N4['CUENTA_x'].astype('str').str[:6])
df_N5['Cuenta_Master'] = np.where(df_N5['largo']==9,df_N5['CUENTA_x'].astype('str').str[:7],df_N5['CUENTA_x'].astype('str').str[:8])
df_N6['Cuenta_Master'] = np.where(df_N6['largo']==11,df_N6['CUENTA_x'].astype('str').str[:9],df_N6['CUENTA_x'].astype('str').str[:10])
df_N7['Cuenta_Master'] = np.where(df_N7['largo']==13,df_N7['CUENTA_x'].astype('str').str[:11],df_N7['CUENTA_x'].astype('str').str[:12])

C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\941484563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_N2['Cuenta_Master'] = np.where(df_N2['largo']==3,df_N2['CUENTA_x'].astype('str').str[:1],df_N2['CUENTA_x'].astype('str').str[:2])
C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\941484563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_N3['Cuenta_Master'] = np.where(df_N3['largo']==5,df_N3['CUENTA_x'].astype('str').str[:3],df_N3['CUENTA_x'].astype('str').str[:4])
C:\Users\jesus\AppData

In [37]:
df_ejemplo

,Descripción,Cuenta regulatoria,Cuenta interna,Sub-cuenta
0,100200102001 Caja,3000,1101,110101
1,100200102001 Caja,3000,1101,110102


In [38]:
df_N2['Cuenta_Master'] = df_N2['Cuenta_Master'].astype('int64')
df_pt_1 = df_N1.merge(df_N2,left_on='CUENTA_x',right_on='Cuenta_Master',how='left')
df_pt_1

C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\2208879785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_N2['Cuenta_Master'] = df_N2['Cuenta_Master'].astype('int64')


,CUENTA_x_x,largo_x,Nivel_x,Cuenta regulatoria_x,Cuenta interna_x,CUENTA_y_x,Descripción_x,CUENTA_x_y,largo_y,Nivel_y,Cuenta regulatoria_y,Cuenta interna_y,CUENTA_y_y,Descripción_y,Cuenta_Master
0,1,1,N1,NaN,NaN,NaN,NaN,121.0,3.0,N2,NaN,NaN,NaN,NaN,1.0
1,1,1,N1,NaN,NaN,NaN,NaN,135.0,3.0,N2,NaN,NaN,NaN,NaN,1.0
2,1,1,N1,NaN,NaN,NaN,NaN,139.0,3.0,N2,NaN,NaN,NaN,NaN,1.0
3,11,2,N1,NaN,NaN,NaN,NaN,1101.0,4.0,N2,3000.0,1101.0,3000.0,100200102001 Caja,11.0
4,11,2,N1,NaN,NaN,NaN,NaN,1102.0,4.0,N2,NaN,NaN,NaN,NaN,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,95,2,N1,NaN,NaN,NaN,NaN,9501.0,4.0,N2,NaN,NaN,NaN,NaN,95.0
208,95,2,N1,NaN,NaN,NaN,NaN,9503.0,4.0,N2,NaN,NaN,NaN,NaN,95.0
209,95,2,N1,NaN,NaN,NaN,NaN,9504.0,4.0,N2,NaN,NaN,NaN,NaN,95.0
210,96,2,N1,NaN,NaN,NaN,NaN,9600.0,4.0,N2,NaN,NaN,NaN,NaN,96.0


In [39]:
df_N2['Cuenta_Master'] = df_N2['Cuenta_Master'].astype('int64')
df_pt_1 = df_N1.merge(df_N2,left_on='CUENTA_x',right_on='Cuenta_Master',how='left')
df_pt_1 = df_pt_1.rename(columns={'CUENTA_x_x':'Cuenta interna','CUENTA_x_y':'Sub-cuenta','Cuenta regulatoria_y':'Cuenta regulatoria','Descripción_y':'Descripción'})
df_pt_1 = df_pt_1[['Descripción','Cuenta regulatoria','Cuenta interna','Sub-cuenta']]

df_N3['Cuenta_Master'] = df_N3['Cuenta_Master'].astype('int64')
df_pt_2 = df_N2.merge(df_N3,left_on='CUENTA_x',right_on='Cuenta_Master',how='left')
df_pt_2 = df_pt_2.rename(columns={'CUENTA_x_x':'Cuenta interna','CUENTA_x_y':'Sub-cuenta','Cuenta regulatoria_y':'Cuenta regulatoria','Descripción_y':'Descripción'})
df_pt_2 = df_pt_2[['Descripción','Cuenta regulatoria','Cuenta interna','Sub-cuenta']]

df_N4['Cuenta_Master'] = df_N4['Cuenta_Master'].astype('int64')
df_pt_3 = df_N3.merge(df_N4,left_on='CUENTA_x',right_on='Cuenta_Master',how='left')
df_pt_3 = df_pt_3.rename(columns={'CUENTA_x_x':'Cuenta interna','CUENTA_x_y':'Sub-cuenta','Cuenta regulatoria_y':'Cuenta regulatoria','Descripción_y':'Descripción'})
df_pt_3 = df_pt_3[['Descripción','Cuenta regulatoria','Cuenta interna','Sub-cuenta']]

df_N5['Cuenta_Master'] = df_N5['Cuenta_Master'].astype('int64')
df_pt_4 = df_N3.merge(df_N5,left_on='CUENTA_x',right_on='Cuenta_Master',how='left')
df_pt_4 = df_pt_4.rename(columns={'CUENTA_x_x':'Cuenta interna','CUENTA_x_y':'Sub-cuenta','Cuenta regulatoria_y':'Cuenta regulatoria','Descripción_y':'Descripción'})
df_pt_4 = df_pt_4[['Descripción','Cuenta regulatoria','Cuenta interna','Sub-cuenta']]

df_N6['Cuenta_Master'] = df_N6['Cuenta_Master'].astype('int64')
df_pt_5 = df_N4.merge(df_N6,left_on='CUENTA_x',right_on='Cuenta_Master',how='left')
df_pt_5 = df_pt_5.rename(columns={'CUENTA_x_x':'Cuenta interna','CUENTA_x_y':'Sub-cuenta','Cuenta regulatoria_y':'Cuenta regulatoria','Descripción_y':'Descripción'})
df_pt_5 = df_pt_5[['Descripción','Cuenta regulatoria','Cuenta interna','Sub-cuenta']]

df_N7['Cuenta_Master'] = df_N7['Cuenta_Master'].astype('int64')
df_pt_6 = df_N5.merge(df_N7,left_on='CUENTA_x',right_on='Cuenta_Master',how='left')
df_pt_6 = df_pt_6.rename(columns={'CUENTA_x_x':'Cuenta interna','CUENTA_x_y':'Sub-cuenta','Cuenta regulatoria_y':'Cuenta regulatoria','Descripción_y':'Descripción'})
df_pt_6 = df_pt_6[['Descripción','Cuenta regulatoria','Cuenta interna','Sub-cuenta']]

C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\1974030877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_N2['Cuenta_Master'] = df_N2['Cuenta_Master'].astype('int64')
C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\1974030877.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_N3['Cuenta_Master'] = df_N3['Cuenta_Master'].astype('int64')
C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\1974030877.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [40]:
df_pt_3

,Descripción,Cuenta regulatoria,Cuenta interna,Sub-cuenta
0,NaN,NaN,110101,NaN
1,NaN,NaN,110103,NaN
2,NaN,NaN,110104,NaN
3,NaN,NaN,110105,NaN
4,NaN,NaN,110106,NaN
...,...,...,...,...
1143,NaN,NaN,960002,96000202.0
1144,NaN,NaN,960101,96010101.0
1145,NaN,NaN,960101,96010102.0
1146,NaN,NaN,960102,96010201.0


In [41]:
DF_FINAL = pd.concat([df_pt_1,df_pt_2,df_pt_3,df_pt_4,df_pt_5,df_pt_6])

In [42]:
# DF_FINAL[Df_total['Cuenta regulatoria'].notna()]

In [43]:
DF_FINAL['Descripción'] = DF_FINAL.groupby('Cuenta interna')['Descripción'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
DF_FINAL['Cuenta regulatoria'] = DF_FINAL.groupby('Cuenta interna')['Cuenta regulatoria'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\1841172722.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  DF_FINAL['Descripción'] = DF_FINAL.groupby('Cuenta interna')['Descripción'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\1841172722.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DF_FINAL['Descripción'] = DF_FINAL.groupby('Cuenta interna')['Descripción'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\jesus\AppData\Local\Temp\ipykernel_38812\1841172722.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill()

In [50]:
df_ejemplo

,Descripción,Cuenta regulatoria,Cuenta interna,Sub-cuenta
0,100200102001 Caja,3000,1101,110101
1,100200102001 Caja,3000,1101,110102


In [49]:
DF_FINAL

,Descripción,Cuenta regulatoria,Cuenta interna,Sub-cuenta
0,NaN,NaN,1,121.0
1,NaN,NaN,1,135.0
2,NaN,NaN,1,139.0
3,100200102001 Caja,3000.0,11,1101.0
4,100200102001 Caja,3000.0,11,1102.0
...,...,...,...,...
1833,NaN,NaN,9501017201,NaN
1834,NaN,NaN,9501023001,NaN
1835,NaN,NaN,9501024802,NaN
1836,NaN,NaN,9501025101,NaN


In [ ]:
DF_FINAL.to_excel(f'C:/Users/jesus/OneDrive/Escritorio/Trabajo/GRUPO SALINAS/Ejercicio_1.xlsx',index = False)